http://www.gisdeveloper.co.kr/?p=8534

https://eunguru.tistory.com/237



tfjob 만드는 과정 => 지나치게 복잡하고 불편한게 현실인듯

1. jupyter에서 단순 학습 테스트 (이 jupyter 파일) (kubeflow jupyter)

2. .py 코드로 만듦 (개발 환경에서) (내 노트북에서)

3. dockerize 시킴 => container가 실행될때 파이썬 코드 실행시키게 만듦 (개발 환경에서) (내 노트북에서)

4. 이미지를 registry에 띄움 (개발 환경에서) (내 노트북에서) (여기서 ECR에 올리고, kubernetes에 secret 넣고 돌릴거 생각하면 힘드니까 개인 dockerhub에 올림)

5. tfjob yaml 파일 작성함 (개발 환경에서) (내 노트북에서)

6. kubectl apply -f 로실행함. (kubernetes terminal에서)

7. yaml파일을 jupyter에서 읽어서 pipeline으로 만듦 (kubeflow jupyter)

#총 3군데 환경을 계속 왔다갔다 해야 함.

#이게 중간에 yaml 파일만 고치면 바로 분산 학습이 될 줄 알았는데 잘 생각해보니까 학습 코드도 고쳐야됨 with strategy 넣어서
#이 과정도 주피터로 돌리고 난 다음 넘어가야지

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import Model

In [3]:
print(tf.__version__)
print(keras.__version__)

2.5.3
2.5.0


In [4]:
(x_train, y_train) , (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [6]:
print('num of machine: {}'.format(strategy.num_replicas_in_sync))

num of machine: 1


In [7]:
#model = keras.models.Sequential([
#]

In [13]:
with strategy.scope():
    input_x = keras.layers.Input(shape=(28,28,1))

    x = Conv2D(32, kernel_size=3, activation='relu')(input_x)
    x = BatchNormalization()(x)
    x = Conv2D(16, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(8, kernel_size=3, activation='relu')(x)

    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_y = Dense(10, activation='softmax')(x)

    model = Model(inputs = input_x, outputs = output_y)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_4 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 16)        4624      
_________________________________________________________________
batch_normalization_5 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 22, 22, 8)         1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 3872)              0   

In [15]:
#0.5코어 주피터라 느리긴 하네
#짜놓고 보니까 valid set을 안넣었네?
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
  25/1875 [..............................] - ETA: 6:41 - loss: 1.5448 - accuracy: 0.4613

KeyboardInterrupt: 

In [38]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 4s - loss: 0.0762 - accuracy: 0.9783 - 4s/epoch - 13ms/step


[0.0761503353714943, 0.9782999753952026]